# QD Loss Example

In Section III.A of our paper, we discussed the method proposed by [Pearce et al. (2018)](https://arxiv.org/pdf/1802.07167.pdf) and argued that *"minimizing $MPIW_{capt}$ does not imply that the width of the PIs generated for the non-captured samples will not decrease along with the width of the PIs generated for the captured samples."* Therefore, the aim of this notebook is to support this claim.

We begin by constructing a toy neural network with four learnable parameters such that $y^u = w_1*x + b_1$ and $y^\ell = w_2*x + b_2$

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

class ToyNet(nn.Module):
    def __init__(self):
        super(ToyNet, self).__init__()
        # Define the learnable parameters with initial values
        self.w1 = nn.Parameter(torch.tensor(1.0), requires_grad=True)
        self.b1 = nn.Parameter(torch.tensor(5.0), requires_grad=True)

        self.w2 = nn.Parameter(torch.tensor(1.5), requires_grad=True)
        self.b2 = nn.Parameter(torch.tensor(0.2), requires_grad=True)

    def forward(self, x):
        # Calculate yu and yl
        yu = self.w1 * x + self.b1
        yl = self.w2 * x + self.b2

        return yu, yl


Recall that the loss function proposed by Pearce et al. is as follows:
$$Loss_{QD} = MPIW_{capt}  + \delta \, \frac{N}{\alpha (1 - \alpha)} \max(0, (1 - \alpha) - PICP) ^ 2$$

where:

$$MPIW_{capt} = \frac{1}{\epsilon + \sum_{i}k_i} \sum_{i=1}^N (\hat{y}^u_i - \hat{y}^\ell_i) \, k_i$$

and:

$$k_i =
    \begin{cases}
      1, & \text{if $\hat{y}^\ell_i < \hat{y}_i < \, \hat{y}^u_i$ and $\hat{y}^\ell_i < y_i < \, \hat{y}^u_i$}\\
      0, & \text{otherwise}.
    \end{cases}$$


In [2]:
def QD_objective(y_pred, y_true, soften_=1, alpha_=0.05, beta_=0.03, device='cuda:0'):
    """Original Loss_QD-soft, adapted from https://github.com/TeaPearce/Deep_Learning_Prediction_Intervals"""
    # Separate upper and lower limits
    y_u = y_pred[0]
    y_l = y_pred[1]

    # Calculate hard captured vector
    K_HU = torch.max(torch.zeros(y_true.size()).to(device), torch.sign(y_u - y_true))
    K_HL = torch.max(torch.zeros(y_true.size()).to(device), torch.sign(y_true - y_l))
    K_H = torch.mul(K_HU, K_HL)

    # Calculate soft captured vector
    K_SU = torch.sigmoid(soften_ * (y_u - y_true))
    K_SL = torch.sigmoid(soften_ * (y_true - y_l))
    K_S = torch.mul(K_SU, K_SL)

    MPIW_c = torch.sum(torch.mul((y_u - y_l), K_H)) / (torch.sum(K_H) + 0.0001)
    PICP_S = torch.mean(K_S)

    # Calculate loss (Eq. 15 QD paper)
    Loss_S = MPIW_c + beta_ * len(y_true) / (alpha_ * (1 - alpha_)) * torch.max(torch.zeros(1).to(device),
                                                                                (1 - alpha_) - PICP_S)
    return Loss_S

Now consider the following toy dataset

In [3]:
# Training data
X = torch.tensor([2, 3, 5, 6])    # Inputs
Y = torch.tensor([4, 6, 10, 12])  # Ground-truth

Given the initial network parameters, when $X$ passes through the network, only three out of four generated PIs cover the target values

In [4]:
# Forward pass
net = ToyNet()
YU, YL = net(X)
YUn, YLn, Yn = np.round(YU.tolist(), 2), np.round(YL.tolist(), 2), np.round(Y.tolist(), 2)
print('Predicted upper bounds YU = ' + str(YUn))
print('Predicted lower bounds YL = ' + str(YLn))
print('----------------------------------------')
print('Targets = ' + str(Yn))
print('----------------------------------------')
print('Covered? k = ' + str((Yn <= YUn) & (YLn <= Yn)))

Predicted upper bounds YU = [ 7.  8. 10. 11.]
Predicted lower bounds YL = [3.2 4.7 7.7 9.2]
----------------------------------------
Targets = [ 4  6 10 12]
----------------------------------------
Covered? k = [ True  True  True False]


Now let's calculate the loss value and perform backpropagation to update the network weights

In [5]:
# Define the optimizer (SGD in this case)
optimizer = optim.SGD(net.parameters(), lr=0.01)

# Calculate the loss using the "QD" loss function
loss = QD_objective([YU, YL], Y, device='cpu')

# Perform the backward pass
optimizer.zero_grad()  # Zero the gradients to avoid accumulation
loss.backward()

# Update the training parameters using SGD
optimizer.step()

# Display the updated parameters
for name, param in net.named_parameters():
    if param.requires_grad:
        print(name, param.data)

w1 tensor(0.9912)
b1 tensor(4.9933)
w2 tensor(1.5178)
b2 tensor(0.2074)


After updating the weights, evaluate the new PIs generated by the network.

**IMPORTANT:**

Note that $k_4 = 0$ (i.e., the fourth sample was not captured initially), and thus its PI width was not considered when calculating the $MPIW_{capt}$ value. However, this did not prevent that the PIs generated for the non-captured sample decreased along with those from the captured samples. The following results indicate that the width PIs of the first three samples were reduced and, notably, the width of the fourth sample was also reduced despite not capturing the corresponding target value.

In [6]:
# New forward pass
YU2, YL2 = net(X)
YU2n, YL2n = np.round(YU2.tolist(), 2), np.round(YL2.tolist(), 2)
print('Original predicted upper bounds YU = ' + str(np.round(YUn.tolist(), 2)))
print('original predicted lower bounds YL = ' + str(np.round(YLn.tolist(), 2)))
print('Targets = ' + str(np.round(Y.tolist(), 2)))
print('----------------------------------------')
print('Covered? k = ' + str((Y <= YU) & (YL <= Y)))
print('\nNew predicted upper bounds YU = ' + str(np.round(YU2n.tolist(), 2)))
print('New predicted lower bounds YL = ' + str(np.round(YL2n.tolist(), 2)))
print('----------------------------------------')
print('Targets = ' + str(np.round(Yn.tolist(), 2)))
print('----------------------------------------')
print('Covered? k = ' + str((Yn <= YUn) & (YLn <= Yn)))

Original predicted upper bounds YU = [ 7.  8. 10. 11.]
original predicted lower bounds YL = [3.2 4.7 7.7 9.2]
Targets = [ 4  6 10 12]
----------------------------------------
Covered? k = tensor([ True,  True,  True, False])

New predicted upper bounds YU = [ 6.98  7.97  9.95 10.94]
New predicted lower bounds YL = [3.24 4.76 7.8  9.31]
----------------------------------------
Targets = [ 4  6 10 12]
----------------------------------------
Covered? k = [ True  True  True False]
